# 📊 Data Preprocessing - Home Credit Default Risk

## 🎯 Цель ноутбука
Этот ноутбук содержит полный процесс предобработки данных для проекта предсказания кредитного дефолта.

## 📋 Содержание:
1. **Загрузка и первичный анализ данных**
2. **Обработка пропущенных значений**
3. **Кодирование категориальных переменных**
4. **Масштабирование числовых признаков**
5. **Сохранение обработанных данных**

---


In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import os

# Настройки для отображения
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("📚 Библиотеки успешно импортированы!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


## 1️⃣ Загрузка и первичный анализ данных


In [ ]:
# Загрузка основных данных
print("📂 Загружаем данные...")

# Основной датасет для обучения
train_df = pd.read_csv('../data_raw/application_train.csv')
print(f"✅ application_train.csv загружен: {train_df.shape}")

# Тестовый датасет
test_df = pd.read_csv('../data_raw/application_test.csv')
print(f"✅ application_test.csv загружен: {test_df.shape}")

# Описание колонок
columns_desc = pd.read_csv('../data_raw/HomeCredit_columns_description.csv')
print(f"✅ HomeCredit_columns_description.csv загружен: {columns_desc.shape}")

print("\n📊 Общая информация о данных:")
print(f"Общее количество записей: {len(train_df):,}")
print(f"Количество признаков: {train_df.shape[1]}")
print(f"Целевая переменная (TARGET): {train_df['TARGET'].value_counts().to_dict()}")
print(f"Процент дефолтов: {train_df['TARGET'].mean()*100:.2f}%")


In [ ]:
# Анализ типов данных
print("🔍 Анализ типов данных:")
print("\nТипы данных:")
print(train_df.dtypes.value_counts())

print("\n📋 Первые 5 строк данных:")
train_df.head()


## 2️⃣ Анализ пропущенных значений


In [ ]:
# Анализ пропущенных значений
print("🔍 Анализ пропущенных значений:")

# Подсчет пропущенных значений
missing_data = train_df.isnull().sum()
missing_percent = (missing_data / len(train_df)) * 100

# Создаем DataFrame с информацией о пропущенных значениях
missing_df = pd.DataFrame({
    'Column': missing_data.index,
    'Missing_Count': missing_data.values,
    'Missing_Percent': missing_percent.values
})

# Сортируем по проценту пропущенных значений
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Percent', ascending=False)

print(f"\n📊 Колонки с пропущенными значениями ({len(missing_df)} из {len(train_df.columns)}):")
print(missing_df.head(20))

# Визуализация пропущенных значений
plt.figure(figsize=(12, 8))
if len(missing_df) > 0:
    top_missing = missing_df.head(15)
    plt.barh(range(len(top_missing)), top_missing['Missing_Percent'])
    plt.yticks(range(len(top_missing)), top_missing['Column'])
    plt.xlabel('Процент пропущенных значений (%)')
    plt.title('Топ-15 колонок с наибольшим количеством пропущенных значений')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print("✅ Пропущенных значений не найдено!")


## 3️⃣ Обработка пропущенных значений


In [ ]:
# Создаем копию данных для обработки
train_processed = train_df.copy()
test_processed = test_df.copy()

print("🔧 Начинаем обработку пропущенных значений...")

# Стратегии обработки пропущенных значений
def handle_missing_values(df, is_train=True):
    """
    Обработка пропущенных значений в датасете
    """
    df_processed = df.copy()
    
    # 1. Колонки с более чем 50% пропущенных значений - удаляем
    high_missing_cols = []
    for col in df_processed.columns:
        if df_processed[col].isnull().sum() / len(df_processed) > 0.5:
            high_missing_cols.append(col)
    
    if high_missing_cols:
        print(f"🗑️ Удаляем колонки с >50% пропущенных значений: {high_missing_cols}")
        df_processed = df_processed.drop(columns=high_missing_cols)
    
    # 2. Числовые колонки - заполняем медианой
    numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
    numeric_missing = [col for col in numeric_cols if df_processed[col].isnull().any()]
    
    if numeric_missing:
        print(f"🔢 Заполняем числовые колонки медианой: {len(numeric_missing)} колонок")
        for col in numeric_missing:
            df_processed[col].fillna(df_processed[col].median(), inplace=True)
    
    # 3. Категориальные колонки - заполняем модой
    categorical_cols = df_processed.select_dtypes(include=['object']).columns
    categorical_missing = [col for col in categorical_cols if df_processed[col].isnull().any()]
    
    if categorical_missing:
        print(f"📝 Заполняем категориальные колонки модой: {len(categorical_missing)} колонок")
        for col in categorical_missing:
            mode_value = df_processed[col].mode()[0] if not df_processed[col].mode().empty else 'Unknown'
            df_processed[col].fillna(mode_value, inplace=True)
    
    return df_processed

# Обрабатываем данные
train_processed = handle_missing_values(train_processed, is_train=True)
test_processed = handle_missing_values(test_processed, is_train=False)

print(f"\n✅ Обработка завершена!")
print(f"Размер train после обработки: {train_processed.shape}")
print(f"Размер test после обработки: {test_processed.shape}")

# Проверяем, остались ли пропущенные значения
remaining_missing_train = train_processed.isnull().sum().sum()
remaining_missing_test = test_processed.isnull().sum().sum()

print(f"\n🔍 Оставшиеся пропущенные значения:")
print(f"Train: {remaining_missing_train}")
print(f"Test: {remaining_missing_test}")


## 4️⃣ Кодирование категориальных переменных


In [ ]:
# Анализ категориальных переменных
print("🔍 Анализ категориальных переменных:")

categorical_cols = train_processed.select_dtypes(include=['object']).columns.tolist()
print(f"Найдено {len(categorical_cols)} категориальных колонок:")
for col in categorical_cols:
    unique_count = train_processed[col].nunique()
    print(f"  - {col}: {unique_count} уникальных значений")

# Кодирование категориальных переменных
def encode_categorical_variables(df_train, df_test):
    """
    Кодирование категориальных переменных с использованием LabelEncoder
    """
    df_train_encoded = df_train.copy()
    df_test_encoded = df_test.copy()
    
    categorical_cols = df_train_encoded.select_dtypes(include=['object']).columns.tolist()
    label_encoders = {}
    
    print(f"\n🔧 Кодируем {len(categorical_cols)} категориальных колонок...")
    
    for col in categorical_cols:
        if col in df_train_encoded.columns and col in df_test_encoded.columns:
            # Создаем LabelEncoder
            le = LabelEncoder()
            
            # Объединяем уникальные значения из train и test для обучения
            all_values = pd.concat([df_train_encoded[col], df_test_encoded[col]]).astype(str)
            le.fit(all_values)
            
            # Применяем кодирование
            df_train_encoded[col] = le.transform(df_train_encoded[col].astype(str))
            df_test_encoded[col] = le.transform(df_test_encoded[col].astype(str))
            
            label_encoders[col] = le
            print(f"  ✅ {col}: {len(le.classes_)} классов")
    
    return df_train_encoded, df_test_encoded, label_encoders

# Применяем кодирование
train_encoded, test_encoded, encoders = encode_categorical_variables(train_processed, test_processed)

print(f"\n✅ Кодирование завершено!")
print(f"Размер train после кодирования: {train_encoded.shape}")
print(f"Размер test после кодирования: {test_encoded.shape}")


## 5️⃣ Масштабирование числовых признаков


In [ ]:
# Подготовка данных для масштабирования
print("🔧 Подготавливаем данные для масштабирования...")

# Исключаем ID и целевую переменную из масштабирования
exclude_cols = ['SK_ID_CURR', 'TARGET']
feature_cols = [col for col in train_encoded.columns if col not in exclude_cols]

print(f"Количество признаков для масштабирования: {len(feature_cols)}")

# Разделяем на числовые и категориальные признаки
numeric_features = train_encoded[feature_cols].select_dtypes(include=[np.number]).columns.tolist()
print(f"Числовых признаков: {len(numeric_features)}")

# Масштабирование с помощью StandardScaler
def scale_features(df_train, df_test, feature_cols):
    """
    Масштабирование числовых признаков
    """
    df_train_scaled = df_train.copy()
    df_test_scaled = df_test.copy()
    
    # Инициализируем StandardScaler
    scaler = StandardScaler()
    
    # Обучаем на train данных
    df_train_scaled[feature_cols] = scaler.fit_transform(df_train[feature_cols])
    
    # Применяем к test данным
    df_test_scaled[feature_cols] = scaler.transform(df_test[feature_cols])
    
    return df_train_scaled, df_test_scaled, scaler

# Применяем масштабирование
train_scaled, test_scaled, scaler = scale_features(train_encoded, test_encoded, numeric_features)

print(f"\n✅ Масштабирование завершено!")
print(f"Размер train после масштабирования: {train_scaled.shape}")
print(f"Размер test после масштабирования: {test_scaled.shape}")

# Проверяем статистики после масштабирования
print(f"\n📊 Статистики после масштабирования (первые 5 числовых признаков):")
print(train_scaled[numeric_features[:5]].describe())


## 6️⃣ Сохранение обработанных данных


In [ ]:
# Сохранение обработанных данных
print("💾 Сохраняем обработанные данные...")

# Создаем директорию для обработанных данных
os.makedirs('../data_processed', exist_ok=True)

# Сохраняем train данные
train_scaled.to_csv('../data_processed/train_processed.csv', index=False)
print("✅ train_processed.csv сохранен")

# Сохраняем test данные
test_scaled.to_csv('../data_processed/test_processed.csv', index=False)
print("✅ test_processed.csv сохранен")

# Сохраняем информацию о признаках
feature_info = {
    'total_features': len(feature_cols),
    'numeric_features': len(numeric_features),
    'categorical_features': len(categorical_cols),
    'feature_names': feature_cols,
    'numeric_feature_names': numeric_features,
    'categorical_feature_names': categorical_cols
}

import json
with open('../data_processed/feature_info.json', 'w') as f:
    json.dump(feature_info, f, indent=2)
print("✅ feature_info.json сохранен")

print(f"\n📊 Итоговая статистика:")
print(f"Исходный размер train: {train_df.shape}")
print(f"Обработанный размер train: {train_scaled.shape}")
print(f"Исходный размер test: {test_df.shape}")
print(f"Обработанный размер test: {test_scaled.shape}")
print(f"Количество признаков: {len(feature_cols)}")
print(f"Целевая переменная (дефолт): {train_scaled['TARGET'].mean()*100:.2f}%")

print(f"\n🎉 Предобработка данных завершена успешно!")
print(f"Обработанные данные сохранены в папке: ../data_processed/")
